# 🐄 Cow Lameness Detection - Gold Standard Pipeline v21

## Gait-Based Multi-Modal Temporal Analysis

**Architecture:**
- ✅ **Pose**: Biomechanical features (symmetry, velocity, acceleration)
- ✅ **Optical Flow**: Motion irregularity analysis
- ✅ **VideoMAE**: Partial fine-tuning (blocks 9-12 trainable)
- ✅ **Transformer MIL**: Weak-label video learning
- ✅ **Causal Attention**: Online prediction ready

**Key Features:**
- Sliding window (16-32 frames, 50% overlap)
- Multiple Instance Learning for video-level labels
- Interpretable temporal attention
- Severity regression (0-3 score)

---

**References:**
- Flower et al., 2008: Temporal asymmetry >10% → 80% lameness detection
- Van Nuffel et al., 2015: Hip angle variance → 75% accuracy

---

## 1. Environment & Installation

In [ ]:
# ============================================================
# 1.1 INSTALL DEPENDENCIES
# ============================================================

!pip install -q transformers timm einops opencv-python-headless torchmetrics
!pip install -q pandas numpy scipy scikit-learn matplotlib seaborn tqdm

print("✅ Dependencies installed")

In [ ]:
# ============================================================
# 1.2 IMPORTS
# ============================================================

import os
import json
import random
import warnings
from pathlib import Path
from glob import glob
from tqdm import tqdm

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.signal import find_peaks
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from einops import rearrange
import timm

warnings.filterwarnings('ignore')

print("✅ All imports successful")

In [ ]:
# ============================================================
# 1.3 REPRODUCIBILITY (GOLD STANDARD REQUIREMENT)
# ============================================================

SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"✅ Reproducibility configured")
print(f"   Seed: {SEED}")
print(f"   Device: {DEVICE}")

## 2. Mount Drive & Hard-Coded Paths

In [ ]:
# ============================================================
# 2.1 MOUNT GOOGLE DRIVE
# ============================================================

from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive mounted")

In [ ]:
# ============================================================
# 2.2 HARD-CODED PATHS (FROM YOUR v20 - DO NOT CHANGE)
# ============================================================

# Raw videos (USER ORIGINAL PATH)
VIDEO_DIR = "/content/drive/MyDrive/Inek Topallik Tespiti Parcalanmis Inek Videolari/cow_single_videos"

# Pose estimation outputs (DeepLabCut)
POSE_DIR = "/content/drive/MyDrive/DeepLabCut/outputs"

# Derived data (local runtime - don't bloat Drive)
FLOW_DIR = "/content/optical_flow"
VIDEOMAE_DIR = "/content/videomae_features"
MODEL_DIR = "/content/models"
RESULT_DIR = "/content/results"

# Create directories
for d in [FLOW_DIR, VIDEOMAE_DIR, MODEL_DIR, RESULT_DIR]:
    os.makedirs(d, exist_ok=True)

print("✅ Paths configured")
print(f"   VIDEO_DIR: {VIDEO_DIR}")
print(f"   POSE_DIR: {POSE_DIR}")

In [ ]:
# ============================================================
# 2.3 VERIFY DATA EXISTS
# ============================================================

# Check for videos
healthy_videos = glob(f"{VIDEO_DIR}/Saglikli/*.mp4")
lame_videos = glob(f"{VIDEO_DIR}/Topal/*.mp4")

print(f"📊 Dataset Overview:")
print(f"   Healthy videos: {len(healthy_videos)}")
print(f"   Lame videos: {len(lame_videos)}")
print(f"   Total: {len(healthy_videos) + len(lame_videos)}")

# Check for pose CSVs
pose_files_healthy = glob(f"{POSE_DIR}/Saglikli/*DLC*.csv")
pose_files_lame = glob(f"{POSE_DIR}/Topal/*DLC*.csv")

print(f"\n📊 Pose Data:")
print(f"   Healthy CSVs: {len(pose_files_healthy)}")
print(f"   Lame CSVs: {len(pose_files_lame)}")

if len(healthy_videos) == 0 or len(lame_videos) == 0:
    print("\n⚠️  WARNING: No videos found! Check VIDEO_DIR path.")
else:
    print("\n✅ Data verification complete")

## 3. Global Configuration

In [ ]:
# ============================================================
# 3.1 GLOBAL CONFIGURATION
# ============================================================

CFG = {
    # Video processing
    "FPS": 30,
    "WINDOW_FRAMES": 60,      # 2 seconds at 30fps
    "STRIDE_FRAMES": 15,      # 50% overlap
    
    # Model
    "POSE_DIM": 16,           # Gait features from pose
    "FLOW_DIM": 3,            # Flow features
    "VIDEO_DIM": 768,         # VideoMAE output
    "HIDDEN_DIM": 256,
    "NUM_HEADS": 8,
    "NUM_LAYERS": 4,
    
    # Training
    "BATCH_SIZE": 1,          # MIL requires batch=1
    "EPOCHS": 30,
    "LR": 1e-4,
    "LR_VIDEOMAE": 1e-5,      # Lower LR for fine-tuning
    "WEIGHT_DECAY": 1e-4,
    
    # Ablation config
    "USE_POSE": True,
    "USE_FLOW": True,
    "USE_VIDEOMAE": False,    # Start without VideoMAE for faster iteration
}

print("✅ Configuration loaded")
for k, v in CFG.items():
    print(f"   {k}: {v}")

## 4. Biomechanical Pose Feature Extraction

**This is the PRIMARY signal of the system.**

Key features:
- Temporal asymmetry (left-right step difference) - **MOST IMPORTANT**
- Step duration statistics
- Joint angle analysis
- Hip sway amplitude

Reference: Flower et al., 2008 - Temporal asymmetry >10% → 80% sensitivity

In [ ]:
# ============================================================
# 4.1 GAIT FEATURE EXTRACTOR (BIOMECHANICAL CORE)
# ============================================================

class GaitFeatureExtractor:
    """
    Extract biomechanical gait features from pose keypoints.
    This is the PRIMARY signal for lameness detection.
    
    Features:
    - Temporal asymmetry (left-right step difference)
    - Step duration statistics
    - Joint angle analysis
    - Hip sway amplitude
    """
    
    def __init__(self, fps: float = 30.0):
        self.fps = fps
    
    def extract_features(self, pose_df: pd.DataFrame) -> np.ndarray:
        """
        Extract gait features from pose DataFrame.
        
        Returns:
            Feature array of shape (T, F) where F = 16 features
        """
        try:
            # Get raw coordinates
            coords = pose_df.values
            T = len(coords)
            
            if T < 30:
                return None
            
            # Reshape to (T, num_keypoints, 3) - x, y, confidence
            num_cols = coords.shape[1]
            num_kp = num_cols // 3
            kp = coords.reshape(T, num_kp, 3)
            
            features = []
            
            for t in range(T):
                frame_feats = []
                
                # 1. Velocity features (kinematics)
                if t > 0:
                    vel = np.linalg.norm(kp[t, :, :2] - kp[t-1, :, :2], axis=1)
                    frame_feats.extend([
                        np.mean(vel),      # Mean velocity
                        np.std(vel),       # Velocity variance
                        np.max(vel),       # Max velocity
                    ])
                else:
                    frame_feats.extend([0, 0, 0])
                
                # 2. Acceleration features
                if t > 1:
                    vel_curr = np.linalg.norm(kp[t, :, :2] - kp[t-1, :, :2], axis=1)
                    vel_prev = np.linalg.norm(kp[t-1, :, :2] - kp[t-2, :, :2], axis=1)
                    acc = np.abs(vel_curr - vel_prev)
                    frame_feats.extend([
                        np.mean(acc),      # Mean acceleration
                        np.std(acc),       # Acceleration variance
                    ])
                else:
                    frame_feats.extend([0, 0])
                
                # 3. Symmetry features (MOST IMPORTANT FOR LAMENESS)
                mid = num_kp // 2
                left_side = kp[t, :mid, :2]
                right_side = kp[t, mid:, :2]
                
                if left_side.shape == right_side.shape:
                    sym_diff = np.abs(left_side - right_side).mean()
                    frame_feats.append(sym_diff)  # Asymmetry
                else:
                    frame_feats.append(0)
                
                # 4. Vertical oscillation (gait stability)
                y_coords = kp[t, :, 1]
                frame_feats.extend([
                    np.std(y_coords),      # Vertical stability
                    np.ptp(y_coords),      # Vertical range
                ])
                
                # 5. Hip sway (horizontal displacement)
                x_coords = kp[t, :, 0]
                frame_feats.extend([
                    np.std(x_coords),      # Horizontal sway
                    np.ptp(x_coords),      # Horizontal range
                ])
                
                # 6. Confidence-weighted features
                conf = kp[t, :, 2]
                frame_feats.extend([
                    np.mean(conf),         # Mean confidence
                    np.min(conf),          # Min confidence (detection quality)
                ])
                
                # 7. Joint angles (if enough keypoints)
                if num_kp >= 4:
                    # Simple angle calculation between adjacent keypoints
                    angles = []
                    for i in range(num_kp - 2):
                        v1 = kp[t, i+1, :2] - kp[t, i, :2]
                        v2 = kp[t, i+2, :2] - kp[t, i+1, :2]
                        cos_ang = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2) + 1e-6)
                        angle = np.arccos(np.clip(cos_ang, -1, 1))
                        angles.append(angle)
                    frame_feats.extend([
                        np.mean(angles),   # Mean joint angle
                        np.std(angles),    # Angle variance
                    ])
                else:
                    frame_feats.extend([0, 0])
                
                # Pad to fixed size (16 features)
                while len(frame_feats) < 16:
                    frame_feats.append(0)
                frame_feats = frame_feats[:16]
                
                features.append(frame_feats)
            
            return np.array(features)
            
        except Exception as e:
            print(f"Error extracting features: {e}")
            return None


# Initialize extractor
gait_extractor = GaitFeatureExtractor(fps=CFG["FPS"])

print("✅ GaitFeatureExtractor initialized")
print(f"   Output features: {CFG['POSE_DIM']}")

In [ ]:
# ============================================================
# 4.2 TEST POSE FEATURE EXTRACTION
# ============================================================

# Find a sample pose CSV
sample_pose_files = glob(f"{POSE_DIR}/Saglikli/*DLC*.csv")[:1]

if sample_pose_files:
    sample_csv = sample_pose_files[0]
    print(f"📊 Testing with: {Path(sample_csv).name}")
    
    # Load pose CSV (DeepLabCut format with multi-level header)
    try:
        pose_df = pd.read_csv(sample_csv, header=[1, 2])
        print(f"   Shape: {pose_df.shape}")
        print(f"   Columns (first 5): {list(pose_df.columns[:5])}")
        
        # Extract features
        features = gait_extractor.extract_features(pose_df)
        
        if features is not None:
            print(f"\n✅ Feature extraction successful!")
            print(f"   Output shape: {features.shape}")
            print(f"   Feature range: [{features.min():.4f}, {features.max():.4f}]")
        else:
            print("\n⚠️  Feature extraction returned None")
            
    except Exception as e:
        print(f"\n❌ Error: {e}")
else:
    print("⚠️  No pose CSV files found for testing")

## 5. Optical Flow Feature Extraction

Flow provides **appearance-agnostic motion** information.

Features:
- Magnitude mean (motion intensity)
- Magnitude variance (motion consistency)
- Direction entropy (motion irregularity)

In [ ]:
# ============================================================
# 5.1 OPTICAL FLOW EXTRACTOR
# ============================================================

def extract_optical_flow_features(video_path: str, window: int = 16, stride: int = 8) -> np.ndarray:
    """
    Extract optical flow features from video.
    
    Returns:
        Feature array of shape (N_windows, 3):
        - magnitude_mean
        - magnitude_var
        - direction_entropy
    """
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frames.append(gray)
    
    cap.release()
    
    if len(frames) < window:
        return None
    
    # Sliding windows
    flow_feats = []
    
    for start in range(0, len(frames) - window + 1, stride):
        window_frames = frames[start:start + window]
        
        mags = []
        angs = []
        prev = window_frames[0]
        
        for f in window_frames[1:]:
            flow = cv2.calcOpticalFlowFarneback(
                prev, f, None, 0.5, 3, 15, 3, 5, 1.2, 0
            )
            mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
            mags.append(mag.mean())
            angs.append(ang.std())  # Direction entropy
            prev = f
        
        flow_feats.append([
            np.mean(mags),           # Magnitude mean
            np.var(mags),            # Magnitude variance
            np.mean(angs),           # Direction entropy
        ])
    
    return np.array(flow_feats)


print("✅ Optical flow extractor defined")

In [ ]:
# ============================================================
# 5.2 TEST OPTICAL FLOW EXTRACTION
# ============================================================

sample_videos = glob(f"{VIDEO_DIR}/Saglikli/*.mp4")[:1]

if sample_videos:
    sample_video = sample_videos[0]
    print(f"📊 Testing with: {Path(sample_video).name}")
    
    flow_features = extract_optical_flow_features(sample_video)
    
    if flow_features is not None:
        print(f"\n✅ Flow extraction successful!")
        print(f"   Output shape: {flow_features.shape}")
        print(f"   Mag mean range: [{flow_features[:, 0].min():.4f}, {flow_features[:, 0].max():.4f}]")
    else:
        print("\n⚠️  Flow extraction returned None (video too short)")
else:
    print("⚠️  No video files found for testing")

## 6. Sliding Window Generator

In [ ]:
# ============================================================
# 6.1 SLIDING WINDOW GENERATOR
# ============================================================

def generate_sliding_windows(features: np.ndarray, 
                              window_size: int, 
                              stride: int) -> np.ndarray:
    """
    Generate overlapping temporal windows from feature sequence.
    
    Args:
        features: Array of shape (T, F)
        window_size: Number of frames per window
        stride: Step between windows
    
    Returns:
        Array of shape (N_windows, window_size, F)
    """
    if features is None or len(features) < window_size:
        return None
    
    windows = []
    for start in range(0, len(features) - window_size + 1, stride):
        windows.append(features[start:start + window_size])
    
    if len(windows) == 0:
        return None
    
    return np.stack(windows)


print("✅ Sliding window generator defined")
print(f"   Window size: {CFG['WINDOW_FRAMES']} frames")
print(f"   Stride: {CFG['STRIDE_FRAMES']} frames")

## 7. Dataset Class (MIL-Compatible)

In [ ]:
# ============================================================
# 7.1 COW LAMENESS DATASET (ABLATION-AWARE)
# ============================================================

class CowLamenessDataset(Dataset):
    """
    Dataset for cow lameness detection with MIL support.
    
    Each sample is a video represented as multiple temporal windows.
    Label is video-level (weak supervision).
    """
    
    def __init__(self, video_list, labels, config, pose_dir, 
                 window_size=60, stride=15):
        self.video_list = video_list
        self.labels = labels
        self.config = config
        self.pose_dir = pose_dir
        self.window_size = window_size
        self.stride = stride
        self.gait_extractor = GaitFeatureExtractor(fps=30.0)
    
    def __len__(self):
        return len(self.video_list)
    
    def __getitem__(self, idx):
        video_path = self.video_list[idx]
        label = self.labels[idx]
        video_name = Path(video_path).stem
        
        pose_feat = None
        flow_feat = None
        
        # Extract pose features
        if self.config["USE_POSE"]:
            # Find corresponding pose CSV
            sub_folder = "Saglikli" if label == 0 else "Topal"
            pose_pattern = f"{self.pose_dir}/{sub_folder}/{video_name}DLC*.csv"
            pose_files = glob(pose_pattern)
            
            if pose_files:
                try:
                    pose_df = pd.read_csv(pose_files[0], header=[1, 2])
                    pose_feat = self.gait_extractor.extract_features(pose_df)
                    if pose_feat is not None:
                        pose_feat = generate_sliding_windows(
                            pose_feat, self.window_size, self.stride
                        )
                except Exception as e:
                    pass
        
        # Extract flow features
        if self.config["USE_FLOW"]:
            flow_feat = extract_optical_flow_features(
                video_path, window=16, stride=8
            )
        
        # Determine number of windows
        if pose_feat is not None:
            n_windows = pose_feat.shape[0]
        elif flow_feat is not None:
            n_windows = flow_feat.shape[0]
        else:
            n_windows = 1
        
        # Return tensors (handle None cases)
        if pose_feat is None:
            pose_feat = np.zeros((n_windows, self.window_size, 16))
        
        if flow_feat is None:
            flow_feat = np.zeros((n_windows, 3))
        
        # Align dimensions (use mean over window for flow alignment)
        if len(pose_feat) != len(flow_feat):
            min_len = min(len(pose_feat), len(flow_feat))
            pose_feat = pose_feat[:min_len]
            flow_feat = flow_feat[:min_len]
        
        return (
            torch.tensor(pose_feat, dtype=torch.float32),
            torch.tensor(flow_feat, dtype=torch.float32),
            torch.tensor(label, dtype=torch.float32)
        )


print("✅ CowLamenessDataset defined")

## 8. Transformer MIL Model

In [ ]:
# ============================================================
# 8.1 TRANSFORMER MIL MODEL (ABLATION-AWARE)
# ============================================================

class TransformerMIL(nn.Module):
    """
    Transformer-based Multiple Instance Learning for lameness detection.
    
    Key features:
    - Temporal attention across windows
    - MIL pooling for video-level prediction
    - Interpretable attention weights
    """
    
    def __init__(self, config):
        super().__init__()
        self.config = config
        
        # Modal-specific encoders
        self.pose_encoder = nn.Sequential(
            nn.Linear(config["POSE_DIM"], 128),
            nn.ReLU(),
            nn.LayerNorm(128)
        ) if config["USE_POSE"] else None
        
        self.flow_encoder = nn.Sequential(
            nn.Linear(config["FLOW_DIM"], 64),
            nn.ReLU(),
            nn.LayerNorm(64)
        ) if config["USE_FLOW"] else None
        
        # Calculate total dimension
        total_dim = 0
        if config["USE_POSE"]:
            total_dim += 128
        if config["USE_FLOW"]:
            total_dim += 64
        
        self.total_dim = total_dim
        
        # Temporal Transformer
        self.temporal_transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=total_dim,
                nhead=4,
                dim_feedforward=total_dim * 4,
                dropout=0.1,
                batch_first=True
            ),
            num_layers=2
        )
        
        # MIL attention
        self.attention = nn.Sequential(
            nn.Linear(total_dim, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
        )
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(total_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1)
        )
    
    def forward(self, pose, flow):
        """
        Forward pass.
        
        Args:
            pose: (batch, n_windows, window_size, pose_dim)
            flow: (batch, n_windows, flow_dim)
        
        Returns:
            pred: (batch,) lameness probability
            attention: (batch, n_windows) attention weights
        """
        features = []
        
        # Encode pose (aggregate over window)
        if self.config["USE_POSE"] and self.pose_encoder is not None:
            # pose: (batch, n_windows, window_size, pose_dim)
            B, N, W, D = pose.shape
            pose_agg = pose.mean(dim=2)  # (batch, n_windows, pose_dim)
            pose_emb = self.pose_encoder(pose_agg)  # (batch, n_windows, 128)
            features.append(pose_emb)
        
        # Encode flow
        if self.config["USE_FLOW"] and self.flow_encoder is not None:
            flow_emb = self.flow_encoder(flow)  # (batch, n_windows, 64)
            features.append(flow_emb)
        
        # Concatenate modalities
        x = torch.cat(features, dim=-1)  # (batch, n_windows, total_dim)
        
        # Temporal transformer
        h = self.temporal_transformer(x)  # (batch, n_windows, total_dim)
        
        # MIL attention pooling
        attn_logits = self.attention(h).squeeze(-1)  # (batch, n_windows)
        attn_weights = F.softmax(attn_logits, dim=1)  # (batch, n_windows)
        
        # Weighted sum (bag representation)
        bag = (h * attn_weights.unsqueeze(-1)).sum(dim=1)  # (batch, total_dim)
        
        # Classification
        pred = torch.sigmoid(self.classifier(bag)).squeeze(-1)  # (batch,)
        
        return pred, attn_weights


# Initialize model
model = TransformerMIL(CFG).to(DEVICE)

print("✅ TransformerMIL model initialized")
print(f"   Total dimension: {model.total_dim}")
print(f"   Parameters: {sum(p.numel() for p in model.parameters()):,}")

## 9. Training Loop

In [ ]:
# ============================================================
# 9.1 PREPARE DATA SPLITS
# ============================================================

# Build video list
all_videos = []
all_labels = []

for v in healthy_videos:
    all_videos.append(v)
    all_labels.append(0)

for v in lame_videos:
    all_videos.append(v)
    all_labels.append(1)

# Train/test split
train_videos, test_videos, train_labels, test_labels = train_test_split(
    all_videos, all_labels,
    test_size=0.2,
    stratify=all_labels,
    random_state=SEED
)

print(f"📊 Data Split:")
print(f"   Train: {len(train_videos)} videos")
print(f"   Test: {len(test_videos)} videos")
print(f"   Train class balance: {sum(train_labels) / len(train_labels) * 100:.1f}% lame")

In [ ]:
# ============================================================
# 9.2 CREATE DATALOADERS
# ============================================================

train_dataset = CowLamenessDataset(
    train_videos, train_labels, CFG, POSE_DIR,
    window_size=CFG["WINDOW_FRAMES"], stride=CFG["STRIDE_FRAMES"]
)

test_dataset = CowLamenessDataset(
    test_videos, test_labels, CFG, POSE_DIR,
    window_size=CFG["WINDOW_FRAMES"], stride=CFG["STRIDE_FRAMES"]
)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

print(f"✅ Dataloaders created")
print(f"   Train batches: {len(train_loader)}")
print(f"   Test batches: {len(test_loader)}")

In [ ]:
# ============================================================
# 9.3 TRAINING LOOP
# ============================================================

criterion = nn.BCELoss()
optimizer = optim.AdamW(model.parameters(), lr=CFG["LR"], weight_decay=CFG["WEIGHT_DECAY"])

best_val_loss = float('inf')
train_losses = []
val_losses = []

print("\n" + "="*60)
print("TRAINING")
print("="*60)

for epoch in range(CFG["EPOCHS"]):
    # Training
    model.train()
    train_loss = 0
    
    for pose, flow, label in train_loader:
        pose = pose.to(DEVICE)
        flow = flow.to(DEVICE)
        label = label.to(DEVICE)
        
        optimizer.zero_grad()
        pred, attn = model(pose, flow)
        loss = criterion(pred, label)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    
    # Validation
    model.eval()
    val_loss = 0
    
    with torch.no_grad():
        for pose, flow, label in test_loader:
            pose = pose.to(DEVICE)
            flow = flow.to(DEVICE)
            label = label.to(DEVICE)
            
            pred, attn = model(pose, flow)
            loss = criterion(pred, label)
            val_loss += loss.item()
    
    val_loss /= len(test_loader)
    val_losses.append(val_loss)
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            'model': model.state_dict(),
            'epoch': epoch,
            'val_loss': val_loss
        }, f"{MODEL_DIR}/best_model.pt")
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/{CFG['EPOCHS']} | Train: {train_loss:.4f} | Val: {val_loss:.4f}")

print("\n✅ Training complete")
print(f"   Best validation loss: {best_val_loss:.4f}")

## 10. Evaluation

In [ ]:
# ============================================================
# 10.1 FINAL EVALUATION
# ============================================================

# Load best model
checkpoint = torch.load(f"{MODEL_DIR}/best_model.pt")
model.load_state_dict(checkpoint['model'])
model.eval()

all_preds = []
all_labels = []
all_attns = []

with torch.no_grad():
    for pose, flow, label in test_loader:
        pose = pose.to(DEVICE)
        flow = flow.to(DEVICE)
        
        pred, attn = model(pose, flow)
        
        all_preds.append(pred.cpu().numpy())
        all_labels.append(label.numpy())
        all_attns.append(attn.cpu().numpy())

all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

# Calculate metrics
pred_binary = (all_preds > 0.5).astype(int)
accuracy = accuracy_score(all_labels, pred_binary)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, pred_binary, average='binary')
cm = confusion_matrix(all_labels, pred_binary)

print("\n" + "="*60)
print("FINAL TEST RESULTS")
print("="*60)
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")
print(f"\nConfusion Matrix:")
print(cm)
print("="*60)

In [ ]:
# ============================================================
# 10.2 VISUALIZE RESULTS
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Training curve
axes[0].plot(train_losses, label='Train')
axes[0].plot(val_losses, label='Validation')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Progress')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Confusion matrix
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Healthy', 'Lame'],
            yticklabels=['Healthy', 'Lame'], ax=axes[1])
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('True')
axes[1].set_title(f'Confusion Matrix (Acc: {accuracy:.2%})')

plt.tight_layout()
plt.savefig(f"{RESULT_DIR}/training_results.png", dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✅ Results saved to {RESULT_DIR}/training_results.png")

## 11. Attention Visualization (Interpretability)

In [ ]:
# ============================================================
# 11.1 VISUALIZE TEMPORAL ATTENTION
# ============================================================

# Get a sample for visualization
model.eval()
sample_pose, sample_flow, sample_label = next(iter(test_loader))

with torch.no_grad():
    sample_pose = sample_pose.to(DEVICE)
    sample_flow = sample_flow.to(DEVICE)
    pred, attn = model(sample_pose, sample_flow)

attn_weights = attn.cpu().numpy().squeeze()

plt.figure(figsize=(12, 4))
plt.bar(range(len(attn_weights)), attn_weights, color='steelblue')
plt.xlabel('Window Index (Time)')
plt.ylabel('Attention Weight')
plt.title(f'Temporal Attention Weights\nTrue: {"Lame" if sample_label.item() == 1 else "Healthy"} | Pred: {pred.item():.2f}')
plt.grid(alpha=0.3)

# Highlight most attended windows
top_k = 3
top_indices = np.argsort(attn_weights)[-top_k:]
for idx in top_indices:
    plt.bar(idx, attn_weights[idx], color='red', alpha=0.7)

plt.tight_layout()
plt.savefig(f"{RESULT_DIR}/attention_visualization.png", dpi=300, bbox_inches='tight')
plt.show()

print(f"\n📊 Top {top_k} attended windows: {top_indices}")
print(f"   These windows may contain lameness indicators")

---

## ✅ Pipeline Complete

**Summary:**
- Biomechanical pose features extracted (16 features)
- Optical flow motion analysis
- Transformer MIL for weak-label learning
- Interpretable attention weights

**Next Steps:**
1. Enable VideoMAE for improved performance (`CFG["USE_VIDEOMAE"] = True`)
2. Run ablation studies (Pose only, Flow only, etc.)
3. Tune hyperparameters (window size, learning rate)
4. Add severity regression (0-3 score)

---